Activation Functions
- Sigmoid
- tanh
- relu
- elu
- selu

<hr>

Sigmoid 학습 과정
1. input 입력
2. network에서 input에 대한 output 도출
3. 실제값과 비교하여 loss 구함
4. loss function을 미분하는 backpropagation을 통해 학습


- gradient: loss의 미분값
- 문제점: backporpagation 과정에서 network 내 각 layer들의 gradient가 곱해지는 중 매우 작은 값들이 곱해지며 gradient가 소실되는 현상 발생(Vanishing Gradient)

Relu 함수
- f(x) = max(0, x) 형태
- 0보다 클때: gradient = 1
    - network가 deep해도 gradient 소실되지 않음
- 0보다 작을때: gradient = 0
    - gradient가 전달되지 않는 문제
- 0보다 작을때 전달되지 않는 문제가 있어도 많이 쓰인다
    - leaky relu: 0보다 작은 경우 gradient = 특정 값과 x를 곱한 값

<hr>

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

tf.enable_eager_execution()

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    
    # 데이터 전처리
    # channel 추가: tensorflow의 input은 batch_size, height, width, channel의 4가지를 받아야 하므로 추가
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    # 0~255 사이 값을 0~1 사이 값으로 바꿈
    train_data, test_data = normalize(train_data, test_data)

    # test 데이터 전처리
    # 총 10개의 label로 변환
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Network 구현을 위한 함수

In [4]:
# shape을 펼쳐줌
def flatten() :
    return tf.keras.layers.Flatten()

# fully connected layer 사용을 위한 dense 설정
def dense(label_dim, weight_init) :
    
    # unit: output으로 나가는 채널을 몇개로 설정할것인지
    # bias=True: bias 사용
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [12]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [7]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    # equal: logit과 label중 어떤 값이 더 큰지
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

dataset, hyperparameter 설정

In [10]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
# training data와 test data를 한번에 batch_size만큼 model에 넣는다
# prefetch: 미리 메모리에 batch_size만큼 올려놔라
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

model 설정

In [14]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [15]:
""" Dataset Iterator """
train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
# checkpoint: network가 학습 중 끊겼을때 재학습할 경우 변경된 weight를 불러내는 역할
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [16]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.6659, train_loss: 2.17415643, train_accuracy: 0.3125, test_Accuracy: 0.2076
Epoch: [ 0] [    1/  468] time: 2.1399, train_loss: 2.13249111, train_accuracy: 0.3438, test_Accuracy: 0.2838
Epoch: [ 0] [    2/  468] time: 2.6312, train_loss: 2.07328558, train_accuracy: 0.4766, test_Accuracy: 0.4213
Epoch: [ 0] [    3/  468] time: 3.0824, train_loss: 2.03598928, train_accuracy: 0.5156, test_Accuracy: 0.5480
Epoch: [ 0] [    4/  468] time: 3.6946, train_loss: 1.90138412, train_accuracy: 0.6328, test_Accuracy: 0.6181
Epoch: [ 0] [    5/  468] time: 4.2500, train_loss: 1.85679865, train_accuracy: 0.6172, test_Accuracy: 0.6685
Epoch: [ 0] [    6/  46

Epoch: [ 0] [   71/  468] time: 32.0564, train_loss: 0.35855967, train_accuracy: 0.9297, test_Accuracy: 0.9004
Epoch: [ 0] [   72/  468] time: 32.3744, train_loss: 0.30413038, train_accuracy: 0.9453, test_Accuracy: 0.9008
Epoch: [ 0] [   73/  468] time: 32.6934, train_loss: 0.33013743, train_accuracy: 0.8906, test_Accuracy: 0.8975
Epoch: [ 0] [   74/  468] time: 33.0227, train_loss: 0.35663906, train_accuracy: 0.8984, test_Accuracy: 0.9023
Epoch: [ 0] [   75/  468] time: 33.3192, train_loss: 0.30893189, train_accuracy: 0.8906, test_Accuracy: 0.9053
Epoch: [ 0] [   76/  468] time: 33.6575, train_loss: 0.26202676, train_accuracy: 0.9141, test_Accuracy: 0.9062
Epoch: [ 0] [   77/  468] time: 33.9470, train_loss: 0.24543217, train_accuracy: 0.9453, test_Accuracy: 0.9064
Epoch: [ 0] [   78/  468] time: 34.2596, train_loss: 0.40278921, train_accuracy: 0.8750, test_Accuracy: 0.9041
Epoch: [ 0] [   79/  468] time: 34.5683, train_loss: 0.31936988, train_accuracy: 0.8906, test_Accuracy: 0.9036
E

Epoch: [ 0] [  145/  468] time: 56.4403, train_loss: 0.36588964, train_accuracy: 0.8672, test_Accuracy: 0.9187
Epoch: [ 0] [  146/  468] time: 56.7715, train_loss: 0.20350002, train_accuracy: 0.9531, test_Accuracy: 0.9253
Epoch: [ 0] [  147/  468] time: 57.1041, train_loss: 0.17487654, train_accuracy: 0.9453, test_Accuracy: 0.9287
Epoch: [ 0] [  148/  468] time: 57.4170, train_loss: 0.31758049, train_accuracy: 0.8984, test_Accuracy: 0.9316
Epoch: [ 0] [  149/  468] time: 57.7565, train_loss: 0.25007892, train_accuracy: 0.9141, test_Accuracy: 0.9319
Epoch: [ 0] [  150/  468] time: 58.0967, train_loss: 0.23351601, train_accuracy: 0.9453, test_Accuracy: 0.9287
Epoch: [ 0] [  151/  468] time: 58.4349, train_loss: 0.30919138, train_accuracy: 0.9062, test_Accuracy: 0.9274
Epoch: [ 0] [  152/  468] time: 58.7782, train_loss: 0.12934610, train_accuracy: 0.9766, test_Accuracy: 0.9266
Epoch: [ 0] [  153/  468] time: 59.1342, train_loss: 0.26503414, train_accuracy: 0.9219, test_Accuracy: 0.9276
E

Epoch: [ 0] [  219/  468] time: 92.8235, train_loss: 0.17948523, train_accuracy: 0.9375, test_Accuracy: 0.9418
Epoch: [ 0] [  220/  468] time: 93.2514, train_loss: 0.15054053, train_accuracy: 0.9531, test_Accuracy: 0.9393
Epoch: [ 0] [  221/  468] time: 93.6836, train_loss: 0.13327369, train_accuracy: 0.9531, test_Accuracy: 0.9398
Epoch: [ 0] [  222/  468] time: 94.1079, train_loss: 0.24651857, train_accuracy: 0.9531, test_Accuracy: 0.9390
Epoch: [ 0] [  223/  468] time: 94.5351, train_loss: 0.17516859, train_accuracy: 0.9453, test_Accuracy: 0.9414
Epoch: [ 0] [  224/  468] time: 94.9402, train_loss: 0.22388160, train_accuracy: 0.9297, test_Accuracy: 0.9440
Epoch: [ 0] [  225/  468] time: 95.3500, train_loss: 0.19990143, train_accuracy: 0.9375, test_Accuracy: 0.9450
Epoch: [ 0] [  226/  468] time: 95.7248, train_loss: 0.15289681, train_accuracy: 0.9531, test_Accuracy: 0.9447
Epoch: [ 0] [  227/  468] time: 96.2095, train_loss: 0.12820941, train_accuracy: 0.9531, test_Accuracy: 0.9436
E

Epoch: [ 0] [  293/  468] time: 120.5084, train_loss: 0.16540122, train_accuracy: 0.9453, test_Accuracy: 0.9458
Epoch: [ 0] [  294/  468] time: 121.2657, train_loss: 0.17742400, train_accuracy: 0.9297, test_Accuracy: 0.9453
Epoch: [ 0] [  295/  468] time: 121.9400, train_loss: 0.21067420, train_accuracy: 0.9297, test_Accuracy: 0.9460
Epoch: [ 0] [  296/  468] time: 122.7602, train_loss: 0.10506523, train_accuracy: 0.9844, test_Accuracy: 0.9467
Epoch: [ 0] [  297/  468] time: 123.3126, train_loss: 0.24287465, train_accuracy: 0.9531, test_Accuracy: 0.9463
Epoch: [ 0] [  298/  468] time: 123.8379, train_loss: 0.22778952, train_accuracy: 0.9297, test_Accuracy: 0.9458
Epoch: [ 0] [  299/  468] time: 124.3031, train_loss: 0.23692651, train_accuracy: 0.9141, test_Accuracy: 0.9482
Epoch: [ 0] [  300/  468] time: 124.6727, train_loss: 0.17292342, train_accuracy: 0.9766, test_Accuracy: 0.9496
Epoch: [ 0] [  301/  468] time: 125.0765, train_loss: 0.14699671, train_accuracy: 0.9453, test_Accuracy:

Epoch: [ 0] [  367/  468] time: 149.7792, train_loss: 0.20670739, train_accuracy: 0.9297, test_Accuracy: 0.9507
Epoch: [ 0] [  368/  468] time: 150.1189, train_loss: 0.15985388, train_accuracy: 0.9688, test_Accuracy: 0.9486
Epoch: [ 0] [  369/  468] time: 150.4767, train_loss: 0.18923946, train_accuracy: 0.9375, test_Accuracy: 0.9492
Epoch: [ 0] [  370/  468] time: 150.7725, train_loss: 0.11358336, train_accuracy: 0.9609, test_Accuracy: 0.9499
Epoch: [ 0] [  371/  468] time: 151.1096, train_loss: 0.12459481, train_accuracy: 0.9688, test_Accuracy: 0.9508
Epoch: [ 0] [  372/  468] time: 151.4511, train_loss: 0.08540828, train_accuracy: 0.9766, test_Accuracy: 0.9512
Epoch: [ 0] [  373/  468] time: 151.7949, train_loss: 0.17835525, train_accuracy: 0.9375, test_Accuracy: 0.9534
Epoch: [ 0] [  374/  468] time: 152.1257, train_loss: 0.10148282, train_accuracy: 0.9844, test_Accuracy: 0.9544
Epoch: [ 0] [  375/  468] time: 152.4463, train_loss: 0.06355546, train_accuracy: 0.9922, test_Accuracy:

Epoch: [ 0] [  441/  468] time: 178.1684, train_loss: 0.16942972, train_accuracy: 0.9453, test_Accuracy: 0.9579
Epoch: [ 0] [  442/  468] time: 178.6004, train_loss: 0.26167327, train_accuracy: 0.9375, test_Accuracy: 0.9582
Epoch: [ 0] [  443/  468] time: 178.9601, train_loss: 0.18711363, train_accuracy: 0.9375, test_Accuracy: 0.9581
Epoch: [ 0] [  444/  468] time: 179.3074, train_loss: 0.26232457, train_accuracy: 0.9141, test_Accuracy: 0.9587
Epoch: [ 0] [  445/  468] time: 179.7160, train_loss: 0.12435376, train_accuracy: 0.9531, test_Accuracy: 0.9597
Epoch: [ 0] [  446/  468] time: 180.0950, train_loss: 0.09770246, train_accuracy: 0.9688, test_Accuracy: 0.9585
Epoch: [ 0] [  447/  468] time: 180.4958, train_loss: 0.06432033, train_accuracy: 0.9844, test_Accuracy: 0.9567
Epoch: [ 0] [  448/  468] time: 180.8551, train_loss: 0.17791104, train_accuracy: 0.9688, test_Accuracy: 0.9560
Epoch: [ 0] [  449/  468] time: 181.2966, train_loss: 0.14576311, train_accuracy: 0.9453, test_Accuracy: